In [577]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import copy

In [11]:
#PATH = '../finanz_vs_news'
PATH = r'..\HackathonDaten\Nachrichten und Kurse\\'
#PATH = ''

In [166]:
#############################################################
#### example how to read the daily stock rates with PATH ####
companyLabel = 'A'    # label of company to load / last column
df_stock = pd.read_csv(PATH+'StocksDaily\ALL_2007-01-01_2017-10-16.csv', parse_dates=["Date"])
StockTickers = df_stock['ticker'].unique()
df_stock = df_stock[df_stock['ticker']=='A']
# save data to pickle
df_stock.to_pickle(PATH+'StocksDaily\stock_'+companyLabel+'.pickle')

In [168]:
# read in data
df_stock = pd.read_pickle(PATH+'StocksDaily\stock_'+companyLabel+'.pickle')

In [712]:
#####################################
#### dictionary for stock labels ####
# get the ticker from our data
df_dummy = pd.read_csv(PATH+'StocksDaily\ALL_2007-01-01_2017-10-16.csv', parse_dates=["Date"])
StockTickers = df_dummy['ticker'].unique()   # get the unique tickers once
# read in dic with all tickers worldwide
TickerAll = pd.read_csv('Yahoo Ticker Symbols - September 2017.csv', delimiter=';')
# delete unneeded columns
del TickerAll['Country']
del TickerAll['Exchange']
# select onyl the ticker of the companies we have
TickerDic = TickerAll.loc[TickerAll['Ticker'].isin(StockTickers)]

namescat = copy.deepcopy(TickerDic['Name'].str.cat(sep = ' '))
namescat = namescat.replace(',' , ' ')
namescat = namescat.replace('.' , ' ')
namescat = namescat.split(' ')
namescat = pd.Series(namescat)
toDel = namescat.value_counts()[:24]
toDel = toDel.index.tolist()

del toDel[7]
del toDel[toDel == ' ']
# take longes ones first, otherwise in replace string one might only replace parts, leaving a rest
toDel.sort(key = lambda x: len(x), reverse=True)

shortName = copy.deepcopy(TickerDic['Name'].values)

for jj, dummy in enumerate(shortName):
    for ii in toDel:
        dummy = dummy.replace(',','')
        dummy = dummy.replace('.','')
        dummy = dummy.replace(' ' + ii,'')
        shortName[jj] = dummy
        
TickerDic['RedName'] = shortName
# save dictionary
TickerDic.to_pickle('TickerDic.pickle')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [717]:
TickerDic.head()

,Ticker,Name,Category Name,RedName
0,AFL,Aflac Incorporated,Accident & Health Insurance,Aflac
1,AIZ,"Assurant, Inc.",Accident & Health Insurance,Assurant
23,UNM,Unum Group,Accident & Health Insurance,Unum
52,IPG,"The Interpublic Group of Companies, Inc.",Advertising Agencies,The Interpublic
57,OMC,Omnicom Group Inc.,Advertising Agencies,Omnicom


In [726]:
# load dictionary
TickerDic = pd.read_pickle('TickerDic.pickle')

In [771]:
#########################################
### LoughranMcDonald MasterDictionary ###
MasterDic = pd.read_csv('LoughranMcDonald_MasterDictionary_2016.csv', delimiter=';')

# get positive words
PosDic = MasterDic[MasterDic['Positive'] !=0]
PosDic = PosDic.sort_values(by=['Word Count'], ascending=False)
PostDic = PosDic['Word']
# save dictionary
PostDic.to_pickle('DicPositiveWords.pickle')

# get negative words
NegDic = MasterDic[MasterDic['Negative'] !=0]
NegDic = NegDic.sort_values(by=['Word Count'], ascending=False)
NegDic = NegDic['Word']
# save dictionary
NegDic.to_pickle('DicNegativeWords.pickle') 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


85081               LOSS
85005             LOSSES
84942        TERMINATION
84917            AGAINST
84829            DEFAULT
84795            CLOSING
84786         IMPAIRMENT
84775            ADVERSE
84704          ADVERSELY
84627            FAILURE
84585         LITIGATION
84582          DISCLOSED
84505         LIMITATION
84479           RESTATED
84377            DAMAGES
84373         TERMINATED
84349      RESTRUCTURING
84348          TERMINATE
84328             BREACH
84321             UNABLE
84319            DECLINE
84307         BANKRUPTCY
84246              FORCE
84236             CLAIMS
84174        LIMITATIONS
84148             UNPAID
84126       DISCONTINUED
84105        LIQUIDATION
84052              FRAUD
84009           CRITICAL
              ...       
32922        DISAPPOINTS
33087        MISLABELLED
33165       QUESTIONABLY
32724       SUSPICIOUSLY
32240         ABRUPTNESS
32098          RIDICULED
31754         DENIGRATED
31631        ABUSIVENESS
31428       RETALIATIONS
